In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from time import time
from unisim import TextSim

2023-10-27 04:38:30.225530: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-27 04:38:30.285846: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loaded backend
Using tf with gpu


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tsim = TextSim(index_type="exact")

2023-10-27 04:38:37.539200: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1636] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78674 MB memory:  -> device: 0, name: NVIDIA H100 80GB HBM3, pci bus id: 0000:18:00.0, compute capability: 9.0
2023-10-27 04:38:37.541376: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1636] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 78674 MB memory:  -> device: 1, name: NVIDIA H100 80GB HBM3, pci bus id: 0000:2a:00.0, compute capability: 9.0
2023-10-27 04:38:37.543423: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1636] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 78674 MB memory:  -> device: 2, name: NVIDIA H100 80GB HBM3, pci bus id: 0000:3a:00.0, compute capability: 9.0
2023-10-27 04:38:37.545273: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1636] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 78674 MB memory:  -> device: 3, name: NVIDIA H100 80GB HBM3, pci bu

UniSim is storing a copy of the indexed data
if you are using large data corpus consider disable this behavior using store_data=False


In [4]:
apa = 'apa' * 1024
g, e = tsim.embed(['2 strings one family'])
print(g.shape, len(e))
g, e = tsim.embed(['2 strings one family', apa])
print(g.shape, len(e))

2023-10-27 04:38:38.812223: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:625] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


(1, 256) 1
(2, 256) 2


<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [5]:
N = 100
ts = time()
for _ in range(N):
    tsim.embed(['2 strings one family'])
total = (time() - ts) * 1000
print(f'total ms {round(total)} - {round(total/N)}ms')

total ms 658 - 7ms


In [9]:
t = 'apa' * 1024
d = tsim.similarity('apapa', t)
print(d)

# golden test 0.48
d = tsim.similarity('Doubting dreams'.lower(), 'rough winds do shake the darling buds of may,'.lower())
print('golden', d)

d = tsim.similarity('I love icecreams', 'I love icecreams')
print('exact', d)

0.26272687315940857
golden 0.4870900809764862
exact 1.0


In [10]:
example = 't' * 4096
print(len(example))


4096


In [12]:
tsim.reset_index()
ts  = time()
chunk = 'cookies'
partial =  'k' * 530 + chunk
tsim.index([example,  partial,  'I love icecreams', 'tea oh tea', 'cookiess!!!', 'no cookies for you'])
tsim.index(['icecream or icecream sandwich that is the question'])

rc = tsim.search(['I love icecreams', 'my icecream', 'cookiess!!!', chunk], pk=2, gk=2)
total = (time() - ts) * 1000
print(f'total ms {round(total)}')
print(rc)
tsim.visualize(rc.results[3])

total ms 75
-[Results Statistics]-
Number of Results: 4
Total Global Matches: 3
Total Partial Matches: 4
Query 3: "cookies"
  idx  is_global    global_sim    is_partial    partial_sim    content
-----  -----------  ------------  ------------  -------------  --------------------------------
    4  True         0.92          True          0.92           cookiess!!!
    5  False        0.8           False                        no cookies for you
    1  False                      True          0.93           kkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkk
